In [1]:
%matplotlib inline
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import pickle

1. Transfer train_v2.csv (raw data) to pickles

In [16]:
in_path = "input/train_v2.csv" #Path where the large file is
#out_path = "" #Path to save the pickle files to
chunk_size = 30000 #size of chunks relies on your available memory
separator = ","

JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

reader = pd.read_csv(in_path,sep=separator,chunksize=chunk_size,
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                    low_memory=False,error_bad_lines=False)    


for i, chunk in enumerate(reader):
    out_file = "input/new_google_{}.pkl".format(i+1)
    with open(out_file, "wb") as f:
        pickle.dump(chunk,f,pickle.HIGHEST_PROTOCOL)

2. Read pickles of whole dataset

In [3]:
import glob
pickle_path = "input" #Same Path as out_path i.e. where the pickle files are

data_p_files=[]
for name in glob.glob(pickle_path + "/new_google_*.pkl"):
    data_p_files.append(name)


df = pd.DataFrame([])
for i in range(len(data_p_files)):
    df = df.append(pd.read_pickle(data_p_files[i]),ignore_index=True)

3. Select 2017 data and parse JSON

In [6]:
df_2017 = df[(df['date']>=20170101) & (df['date']<=20171231)].reset_index(drop=True)

In [7]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
for column in JSON_COLUMNS:
    column_as_df = json_normalize(df_2017[column])
    column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
    df_2017 = df_2017.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
print(f"Extracted df_2017. Shape: {df_2017.shape}")

Extracted df_2017. Shape: (928860, 59)


In [8]:
df_2017.head()

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,Firefox,...,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Chrome,...,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Chrome,...,NaN,not available in demo dataset,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,9075655783635761930,"[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,1508169851,1,1508169851,Chrome,...,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,6960673291025684308,"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,1508190552,1,1508190552,Chrome,...,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


4. Drop some columns

In [10]:
df_2017 = df_2017.drop(['hits', 'customDimensions'], axis=1)

In [12]:
# select constant columns
const_cols = [c for c in df_2017.columns if df_2017[c].nunique(dropna=False)==1 ]
const_cols

['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.operatingSystemVersion',
 'device.mobileDeviceBranding',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.flashVersion',
 'device.language',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [14]:
df_2017 = df_2017.drop(const_cols, axis=1)

In [15]:
df_2017.shape

(928860, 38)

In [16]:
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928860 entries, 0 to 928859
Data columns (total 38 columns):
channelGrouping                                 928860 non-null object
date                                            928860 non-null int64
fullVisitorId                                   928860 non-null object
visitId                                         928860 non-null int64
visitNumber                                     928860 non-null int64
visitStartTime                                  928860 non-null int64
device.browser                                  928860 non-null object
device.operatingSystem                          928860 non-null object
device.isMobile                                 928860 non-null bool
device.deviceCategory                           928860 non-null object
geoNetwork.continent                            928860 non-null object
geoNetwork.subContinent                         928860 non-null object
geoNetwork.country                         

In [17]:
df_2017.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,...,trafficSource.medium,trafficSource.keyword,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,Windows,False,desktop,...,organic,water bottle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,Chrome OS,False,desktop,...,referral,NaN,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,Android,True,mobile,...,(none),NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,Windows,False,desktop,...,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,Windows,False,desktop,...,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


5. Save 2017 data to pickle

In [19]:
df_2017.to_pickle("2017_clean.pkl")